In [18]:
from transformers import AutoTokenizer
from eval_dataset import FarsInstructEvalDataset

# checkpoint = "/media/abbas/Backup/mGPT-1.3B-persian/"
# checkpoint = "/media/abbas/Backup/aya"
# checkpoint = "/media/abbas/Backup/PersianMind-v1.0"
# checkpoint = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/base_checkpoints/ava-llama3-v2/"
# checkpoint = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/lora_checkpoints/hf_ckpt_ava_raw_fine_tune_with_really_small_samples"
# checkpoint = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/lora_checkpoints/hf_ckpt_micro_ava_sajjadqa--qacatg-titlgen-genqWRa_pnsum--sumartcl-whatctgblngsto_slpl--genqwitha-qora_wikisum--artclsum-gnralans_digi--whichctg_persiannews--chscatg_parssentiment--revasp-revcat_exap--rel-ordr_absa--plr_pner--fndprsn"
path = "/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/FarsInstruct/data/1shot_instruct_dataset_test_all.csv"
tokenizer = AutoTokenizer.from_pretrained(checkpoint, padding_side="left")

ds = FarsInstructEvalDataset(tokenizer, 129, 'none', 'test', shots=1, path=path)
encoded_ds = ds.get_tokenized_data('PNLPhub/snappfood-sentiment-analysis', 'what_is_sentiment', multiple_choice=False)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [19]:
encoded_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

In [20]:
print(tokenizer.decode(encoded_ds['input_ids'][0]))
print(tokenizer.decode(encoded_ds['attention_mask'][0]))
print(tokenizer.decode(encoded_ds['labels'][0]))

<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>کاربری پس از خرید یک محصول نظر زیر را در مورد آن دارد. بررسی کن که آیا او از خریدش خوشحال است یا ناراحت؟

نظر: غذا خیلی سرد تحویل گرفتم، گذاشتمش کنار، نمیشه خورد
احساس کاربر:
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!""""""""""""""""""""""""""""""""""""""""""""

In [21]:
from transformers import DataCollatorWithPadding
from torch.utils.data import DataLoader

data_collator = DataCollatorWithPadding(tokenizer,
                                        return_tensors='pt')
        
val_dataloader = DataLoader(encoded_ds, collate_fn=data_collator, batch_size=32)

In [22]:
from transformers import AutoModelForCausalLM, AutoModelForSeq2SeqLM
from accelerate import Accelerator
import torch


device = "cuda" if torch.cuda.is_available() else "cpu"

from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

try:
    model = AutoModelForCausalLM.from_pretrained(checkpoint,
                                                #quantization_config=quantization_config,
                                                torch_dtype=torch.float16,
                                                device_map="auto"
    )
except:
    model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint,
                                                quantization_config=quantization_config,
                                                torch_dtype=torch.float16,
                                                device_map="auto"
    )

accelerator = Accelerator(cpu=False)

val_dataloader, model = accelerator.prepare(val_dataloader, model)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [23]:
model.name_or_path

'/mnt/beegfs/wrkdir/u111187/Hojjat_Workstation/FarsInstruct/base_checkpoints/ava-llama3-v2/'

In [24]:
import torch 
import numpy as np
import datasets
from hazm import sent_tokenize


def postprocess_text(preds, labels):
        preds = [pred.strip() for pred in preds]
        labels = [label.strip() for label in labels]

        # rougeLSum expects newline after each sentence
        preds = ["\n".join(sent_tokenize(pred)) for pred in preds]
        labels = ["\n".join(sent_tokenize(label)) for label in labels]

        return preds, labels


def evall():
    dec_preds = []
    dec_labels = []
    model.eval()
    metric = datasets.load_metric('rouge')
    for step, batch in enumerate(val_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                input_ids=batch['input_ids'],
                attention_mask=batch['attention_mask'],
                max_new_tokens=64,
                top_k=10,
                # eos_token_id=128001
            )

            generated_tokens = accelerator.pad_across_processes(
                generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
            )
            labels = batch["labels"]
            
            generated_tokens, labels = accelerator.gather_for_metrics((generated_tokens, labels))
            generated_tokens = generated_tokens.cpu().numpy()
            labels = labels.cpu().numpy()

            labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
            if isinstance(generated_tokens, tuple):
                generated_tokens = generated_tokens[0]
            decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
            if model.name_or_path != "/media/abbas/Backup/aya":
                decoded_preds = tokenizer.batch_decode(generated_tokens[:, batch['input_ids'].shape[1]:], skip_special_tokens=True)
            else:
                decoded_preds = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
                
            decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
            metric.add_batch(
                predictions=decoded_preds,
                references=decoded_labels,
            )

            # print(decoded_preds)
            # print(tokenizer.batch_decode(batch['input_ids']))

            dec_preds = decoded_preds
            dec_labels = decoded_labels

    scores = metric.compute(
        rouge_types=["rouge1", "rouge2", "rougeL", "rougeLsum"], 
        use_stemmer=False, 
        lang='fa')

    from prettytable import PrettyTable
    table = PrettyTable(title='Rouge Metric')
    table.field_names = ["Score", "Type", "Precision", "Recall", "F-Measure"]
    for i, score_name in enumerate(scores):
        score = dict(scores[score_name]._asdict())
        for score_type in score:
            values = score[score_type]
            table.add_row([f'{score_name}', f'{score_type}', f'{values.precision * 100:0.2f} ', f'{values.recall * 100:0.2f}', f'{values.fmeasure * 100:0.2f}'])

        if i < len(scores) - 1:
            table.add_row([''] * len(table.field_names))
            table.add_row(['***'] * len(table.field_names))
            table.add_row([''] * len(table.field_names))

    rouge_table_valid = table
    print(table.get_string())

    return dec_preds, dec_labels
    # result = {k: round(v * 100, 4) for k, v in result.items()}


## Snappfood sentiment

In [25]:
# ava -- raw
evall()

/home/u111187/.local/lib/python3.10/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/u111187/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   38.30   | 75.00  |   41.27   |
|   rouge1  | mid  |   47.99   | 83.00  |   50.79   |
|   rouge1  | high |   56.53   | 90.00  |   59.23   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   39.00   | 75.00  |   41.94   |
|   rougeL  | mid  |   48.09

(['ناراحت دلیل: نان دسر کاملا خشک و مانده بود.',
  'ناراحت',
  'ونم از این مغازه.',
  'ناراحت'],
 ['ناراحت', 'خوشحال', 'خوشحال', 'ناراحت'])

In [14]:
## ava -- fine tuned -- <|temp|>
evall()

/tmp/ipykernel_126378/618092861.py:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('rouge')
/home/u111187/.local/lib/python3.10/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[2024-06-11 23:35:11,635] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


/home/u111187/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   14.07   | 43.00  |   19.29   |
|   rouge1  | mid  |   18.49   | 52.00  |   25.20   |
|   rouge1  | high |   23.12   | 61.00  |   31.29   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   13.72   | 42.98  |   18.85   |
|   rougeL  | mid  |   18.48

(['خوشحال نیستم',
  'خوشحال نیستم از خریدم.\nبرای من مناسب نبود.\nبرای کسانی که لاکتوز تحملشون خوبه خوبه.\nبرای منم که تحملشون متوسط بود خیلی مزه ی بد و نامطبوع داشت.\nبا این قیمت حتما باید خیلی بهتر از این باش',
  'من هم از خریدش راضی هستم.\nخوب بود، فقط من میخواستم یک تن ماهی خشک سفارش بدم ولی گفتن ماهی خشک ندارن، من دیگه از دیجی کالا ماهی نمیخرم.\nممنون از دیجی کالا.\nممنون از',
  'خوشحال نیستم!'],
 ['ناراحت', 'خوشحال', 'خوشحال', 'ناراحت'])

## Query paraphrase -- similar or not

In [16]:
# ava -- raw
evall()

/home/u111187/.local/lib/python3.10/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/u111187/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   1.82    | 41.40  |    3.25   |
|   rouge1  | mid  |   2.55    | 46.00  |    4.01   |
|   rouge1  | high |   3.58    | 50.20  |    5.07   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   1.82    | 41.40  |    3.21   |
|   rougeL  | mid  |   2.57 

(['صحیح متن اول و دوم هم معنی هستند.\nهر دو سؤال از کجا باید برود تا شب را تنها بگذارد، اما با استفاده از کلمات و عبارات مختلف.',
  'صحیح متن اول و دوم هم معنی هستند.\nهر دو از تی ام بکس دانلود اهنگ کی بودی چی بودی تو را درخواست می کنند.',
  'صحیح، دو متن هم معنی هستند.\nآنها هر دو سؤال می کنند که کیسه صفرا در کجای بدن قرار دارد، اما متن دوم سؤال را به صورت یک جمله بیان می کند.',
  'صحیح، دو متن هم معنی هستند.\nهر دو درباره غذاهایی هستند که می توان در دستگاه های خاصی مانند توستر و ساندویچ ساز درست کرد.',
  'غلط متن اول در مورد راهکارهای درمان افسردگی و متن دوم در مورد روش\u200cهای مهاجرت به آمریکا است.\nاین دو موضوع کاملاً متفاوت هستند و هیچ ارتباطی با هم ندارند.\nبنابراین، این دو متن هم معنی نیستند.',
  'متن اول و دوم هم معنی نیستند.\nمتن اول سؤال می کند که چه مدت طول می کشد تا بتوانیم رآکتور هسته ای را جدا کنیم، در حالی که متن دوم سؤال می کند که ساخت یک رآکتور هسته ای چقدر طول می کشد.\nاین دو',
  'صحیح، دو متن هم معنی هستند.\nهر دو در مورد نحوه تشخیص اینکه آیا یک حساب کاربری در تلگرا

## Query paraphrase -- compare two sents

In [ ]:
# 

In [8]:
# ava -- raw
evall()

/tmp/ipykernel_124966/1123409467.py:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('rouge')
/home/u111187/.local/lib/python3.10/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[2024-06-11 22:33:02,917] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


/home/u111187/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   2.51    | 57.00  |    4.73   |
|   rouge1  | mid  |   2.81    | 61.40  |    5.27   |
|   rouge1  | high |   3.10    | 65.60  |    5.78   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   2.51    | 56.80  |    4.72   |
|   rougeL  | mid  |   2.80 

(['بله، سوال دوم از سوال اصلی بازنویسی شده است.\nهر دو سوال از کجا باید برم برای یک شب را می پرسند، اما سوال دوم از کلمه "منو تنها" به "منو راحت" تغییر می کند.\nاین تغییر به طور کلی به منظور ایجاد یک سوال',
  'بله، سوال دوم از سوال اصلی بازنویسی شده است.',
  'بله، سوال دوم از سوال اصلی بازنویسی شده است.',
  'بله، سوال دوم از سوال اصلی بازنویسی شده است.\nهر دو سوال در مورد غذاهای مختلفی است که می توان در یک وسیله خاص (توستر یا ساندویچ ساز) درست کرد.',
  'خیر، سوال دوم از سوال اصلی بازنویسی نشده است.\nسوال اصلی در مورد راهکارهای درمان افسردگی است، در حالی که سوال دوم در مورد روش\u200cهای مهاجرت به آمریکا است.',
  'بله، سوال دوم از سوال اصلی بازنویسی شده است.\nهر دو سوال در مورد ساخت یک رآکتور هسته ای هستند، اما سوال دوم به طور مستقیم از سوال اصلی بازنویسی شده است.',
  'بله، سوال دوم از سوال اصلی بازنویسی شده است.\nهر دو سوال از کجا می توانیم متوجه شیم که در یک پلتفرم پیام رسانی هک شده ایم، با این تفاوت که سوال دوم به واتساپ اشاره دارد در حالی که سوال اصلی به تلگر',
  'بله، سوال دوم از سو

In [23]:
# ava -- fine tuned with 'none' temp
evall()

/home/u111187/.local/lib/python3.10/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/u111187/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   21.71   | 56.60  |   29.89   |
|   rouge1  | mid  |   23.54   | 60.80  |   32.29   |
|   rouge1  | high |   25.48   | 64.80  |   34.84   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   21.47   | 56.60  |   29.63   |
|   rougeL  | mid  |   23.59

(['بله بازنویسی شده است.',
  'دانلود اهنگ کی بودی تو از تی ام بکس.\nبله',
  'سؤال دوم از سوال اصلی بازنویسی شده است.\nخیر',
  'بله خیر',
  'بله خیر',
  'بله خیر',
  'بله خیر',
  'بله خیر',
  'مواد غذای برای کبد چرب مفید است.\nخیر',
  'برای بهبود مکالمه زبان انگلیسی من چه کاری می توانم انجام دهم؟\nسوال های مرتبط: چگونه می توانم در انگلیسی بهتر شوم؟\nچگونه می توانم انگلیسی را بهتر یاد بگیرم؟\nچگونه می توانم انگلیسی را بهتر صحبت کنم؟\nچگونه می توانم انگلیسی را بهتر بنوی',
  'پروتئین در میوه های انگور و انگور است.\nمقایسه: بله',
  'بله خیر',
  'بله خیر',
  '۱ ۲ ۳ ۴ ۵ ۶ ۷ ۸ ۹ ۱۰ ۱۱ ۱۲ ۱۳ ۱۴ ۱۵ ۱۶ ۱۷ ۱۸ ۱۹ ۲۰ ۲۱ ۲۲ ۲۳ ۲۴ ۲۵ ۲۶ ۲۷ ۲۸ ۲۹ ۳۰ ۳۱',
  'چگونه می توان عکس ها را از رایانه به آیفون انتقال داد؟\nچگونه می توان iTunes را برای همگام سازی عکس ها از رایانه شخصی با آیفون خود تنظیم کرد؟\nبا استفاده از برنامه های دیگر.\nبا استفاده از برنامه های دیگر.\nبا استفاده از برنامه های دیگر',
  'بعد از کشیدن دندان دهان بو میدهد.\nتائید شده؟\n/  غیر تائید شده غیر تائید شده',
  'بله جواب:  خیر',
  'بله با

In [15]:
# ava -- fine tuned with temp
evall()

/home/u111187/.local/lib/python3.10/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/u111187/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   0.38    | 34.00  |    0.74   |
|   rouge1  | mid  |   0.45    | 38.00  |    0.88   |
|   rouge1  | high |   0.53    | 42.60  |    1.04   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   0.37    | 34.00  |    0.73   |
|   rougeL  | mid  |   0.45 

(['خیر<|end_of_text|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>',
  'بله<|end_of_text|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_

## Farstail -- confidence

In [9]:
# ava -- finetuend
evall()

/tmp/ipykernel_121202/3065205785.py:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('rouge')
/home/u111187/.local/lib/python3.10/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[2024-06-11 19:55:38,912] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


/home/u111187/.local/lib/python3.10/site-packages/bitsandbytes/nn/modules.py:426: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   0.17    | 28.25  |    0.33   |
|   rouge1  | mid  |   0.19    | 32.25  |    0.38   |
|   rouge1  | high |   0.21    | 36.00  |    0.43   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  1.00  |    0.00   |
|   rouge2  | high |   0.01    |  2.50  |    0.01   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   0.17    | 28.75  |    0.33   |
|   rougeL  | mid  |   0.19 

(['سطح اطمینان خود را در شباهت عبارات ارائه شده بیان کنید\nعبارت اول: از نگاه و نگرش شهید آیت الله صدر، انقلاب جنبشی سرسختانه و بر مبنای اصول مکتبی معینی است که با درک واقعیت\u200cهای موجود به دنبال تغییر ودگرگونی بنیادین در تمامی ابعاد زندگی فردی، اجتماعی و سیاسی و معیارها، اصول و ارزش\u200cهای حاکم است.\nعبارت دوم: انقلاب از نظر شهید صدر جنبشی سرسختانه و برپایه اصول مکتب معینی است.\nسطح اطمینان: اول عبارت اول: از نگاه و نگرش شهید آیت الله صدر، انقلاب جنبشی سرسختانه و بر مبنای اصول مکتبی معینی است که با درک واقعیت\u200cهای موجود به دنبال تغییر ودگرگونی بنیادین در تمامی ابعاد زندگی فردی، اجتماعی و سیاسی و معیارها، اصول و ارزش\u200cهای حاکم است.\nعبارت دوم: انقلاب از نظر شهید صدر جنبشی سرسختانه و برپایه اصول مکتب معینی است.\nسطح اطمینان: دوم�لدانی: در این فیلم چه کسی نقش اصلی را بازی می\u200cکند؟\nجواب: مایکل جکسون در فیلم ویتنییز (Wiz) مایکل جکسون نقش اصلی را بازی می\u200cکند.\nویتنییز (Wiz) یک برنامه تلویزیونی آمریکایی است که در سال ۱۹۷۸ میلادی برای شبکه NBC تهیه و تولید شد.\nاین برنا

## Farstail -- can you infer

In [8]:
# ava -- continual-cola with temp
evall()

/tmp/ipykernel_124088/393536001.py:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('rouge')
/home/u111187/.local/lib/python3.10/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


[2024-06-11 21:44:32,920] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.
 [WARNING]  async_io: please install the libaio-devel package with yum
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  NVIDIA Inference is only supported on Ampere and newer architectures
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.2
 [WARNING]  using untested triton version (2.2.0), only 1.0.0 is known to be compatible


/home/u111187/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `10` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   1.36    | 36.40  |    2.55   |
|   rouge1  | mid  |   1.66    | 40.60  |    3.02   |
|   rouge1  | high |   2.05    | 45.00  |    3.61   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.10    |  0.80  |    0.17   |
|   rouge2  | mid  |   0.33    |  2.00  |    0.53   |
|   rouge2  | high |   0.67    |  3.20  |    1.01   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   1.36    | 36.20  |    2.55   |
|   rougeL  | mid  |   1.68 

(['بله<|end_of_text|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>',
  'بله<|end_of_text|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>',
  'بله<|end_of_text|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>',
  'بله<|end_of_text|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>',
  'بله<|end_of_text|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|i

In [10]:
# ava -- continual-cola
evall()


/home/u111187/.local/lib/python3.10/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   1.18    | 66.12  |    2.28   |
|   rouge1  | mid  |   1.42    | 72.50  |    2.73   |
|   rouge1  | high |   1.71    | 78.75  |    3.26   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.44    | 18.00  |    0.85   |
|   rouge2  | mid  |   0.62    | 23.50  |    1.19   |
|   rouge2  | high |   0.78    | 29.50  |    1.51   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   1.18    | 66.50  |    2.29   |
|   rougeL  | mid  |   1.42 

(['- بله <|im_start|>گزینه های دیگر: - خیر   نمیتوان نظر قطعی داد   - بله<|end_of_text|><|start_header_id|><|start_header_id|>assistant: What is the difference between a manager and a leader?\nManager: A manager is someone who is responsible for planning, organizing, and controlling resources to achieve specific goals and objectives.\nThey are typically focused on efficiency, productivity, and effectiveness.\nManagers are often responsible for tasks such as budgeting, scheduling, and monitoring progress.\nThey may also be responsible for delegating tasks to team members and providing guidance and support.\nLeader: A leader is someone who inspires and motivates others to work towards a',
  '- بله <|im_start|>گزینه اول درست است.<|end_of_text|><|start_header_id|><|start_header_id|><|start_header_id|><|start_header_id|><|start_header_id|><|start_header_id|><|start_header_id|><|start_header_id|><|start_header_id|><|start_header_id|><|start_header_id|><|start_header_id|><|start_header_id|><|

In [9]:
# aya
evall()

/tmp/ipykernel_2425540/1637929580.py:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('rouge')
/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   38.00   | 38.00  |   38.00   |
|   rouge1  | mid  |   45.50   | 45.50  |   45.50   |
|   rouge1  | high |   52.50   | 52.50  |   52.50   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   39.00   | 39.00  |   39.00   |
|   rougeL  | mid  |   45.50

(['بله', 'بله', 'بله', 'بله', 'بله', 'خیر.', 'خیر.', 'خیر.'],
 ['خیر',
  'نمیتوان نظر قطعی داد',
  'بله',
  'نمیتوان نظر قطعی داد',
  'بله',
  'نمیتوان نظر قطعی داد',
  'نمیتوان نظر قطعی داد',
  'خیر'])

In [46]:
# persian_mind
evall()

/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   12.79   | 28.50  |   17.37   |
|   rouge1  | mid  |   15.71   | 34.00  |   21.24   |
|   rouge1  | high |   19.00   | 41.00  |   25.62   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   12.58   | 27.50  |   16.97   |
|   rougeL  | mid  |   15.63

(['خیر</s>',
  'خیر</s>',
  'ب</s>',
  'خیر</s>',
  'خیر</s>',
  'خیر</s>',
  'خیر</s>',
  'خیر</s>'],
 ['خیر',
  'نمیتوان نظر قطعی داد',
  'بله',
  'نمیتوان نظر قطعی داد',
  'بله',
  'نمیتوان نظر قطعی داد',
  'نمیتوان نظر قطعی داد',
  'خیر'])

In [54]:
# mgpt 
evall()

/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Set

+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   0.19    | 23.75  |    0.38   |
|   rouge1  | mid  |   0.24    | 30.12  |    0.48   |
|   rouge1  | high |   0.30    | 36.50  |    0.58   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   0.19    | 23.75  |    0.38   |
|   rougeL  | mid  |   0.25 

(['بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\n- بله<|endoftext|>\

In [ ]:
# tk-instruct

In [61]:
# ava
evall()

/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   1.66    | 44.38  |    2.75   |
|   rouge1  | mid  |   2.62    | 50.88  |    4.25   |
|   rouge1  | high |   3.81    | 57.50  |    6.08   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   1.67    | 43.50  |    2.77   |
|   rougeL  | mid  |   2.58 

(['بله<|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|><|im_end|>',
  'بله.\nبر اساس عبارت اول، نثر مرسل نوعی نثر است که ساده و از پیرایه\u200cها و آ

In [12]:
# ava with <|temp|>
evall()

/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   2.24    | 100.00 |    4.31   |
|   rouge1  | mid  |   2.50    | 100.00 |    4.82   |
|   rouge1  | high |   2.78    | 100.00 |    5.34   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   1.07    | 28.50  |    2.05   |
|   rouge2  | mid  |   1.30    | 35.00  |    2.51   |
|   rouge2  | high |   1.57    | 41.51  |    3.02   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   2.24    | 100.00 |    4.33   |
|   rougeL  | mid  |   2.51 

(['آیا براساس عبارت اول میتوان عبارت دوم را استنتاج کرد؟\nجواب را از بین گزینه های داده شده انتخاب کن - بله - خیر - نمیتوان نظر قطعی داد\nعبارت اول: بُرهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nعبارت دوم: برهان فطرت نام دیگر برهان اتقان صنع است.\nجواب: <im_start>',
  'آیا براساس عبارت اول میتوان عبارت دوم را استنتاج کرد؟\nجواب را از بین گزینه های داده شده انتخاب کن - بله - خیر - نمیتوان نظر قطعی داد\nعبارت اول: نثر مرسل که با نام نثر ساده نیز عنوان می شود، نوشته\u200cای است ساده و از پیرایه\u200cها و آرایش\u200cهای ادبی خالی است.\nعبارت دوم: نوع نثر تاریخ بیهقی، نثر مرسل\u200c است.\nجواب: <im_start>',
  'آیا براساس عبارت اول میتوان عبارت دوم را استنتاج کرد؟\nجواب را از بین گزینه های داده شده انتخاب کن - بله - خیر - نمیتوان نظر قطعی داد\nعبارت اول: رودابه نام دختر مهراب کابلی و سیندخت است او همسر زال و مادر رستم(شخصیت اسطوره ای شاهنامه) می\u200cباشد.\nزال در گذری از مرز کابل، به رودابه علاقه\u200cمند شد و او را به 

In [14]:
# ava with temp
evall()

/tmp/ipykernel_2429477/3065205785.py:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('rouge')
/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/hom

+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   1.43    | 100.00 |    2.81   |
|   rouge1  | mid  |   1.47    | 100.00 |    2.90   |
|   rouge1  | high |   1.52    | 100.00 |    3.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   1.43    | 100.00 |    2.81   |
|   rougeL  | mid  |   1.47 

(['Human:آیا جمله دوم به جمله اول مرتبط است؟\nصحیح، غلط، شاید\nپیش فرض: بُرهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nفرضیه: برهان فطرت نام دیگر برهان اتقان صنع است.\nجواب: <im_start>Assistant:',
  'Human:آیا جمله دوم به جمله اول مرتبط است؟\nصحیح، غلط، شاید\nپیش فرض: نثر مرسل که با نام نثر ساده نیز عنوان می شود، نوشته\u200cای است ساده و از پیرایه\u200cها و آرایش\u200cهای ادبی خالی است.\nفرضیه: نوع نثر تاریخ بیهقی، نثر مرسل\u200c است.\nجواب: <im_start>Assistant:',
  'Human:آیا جمله دوم به جمله اول مرتبط است؟\nصحیح، غلط، شاید\nپیش فرض: رودابه نام دختر مهراب کابلی و سیندخت است او همسر زال و مادر رستم(شخصیت اسطوره ای شاهنامه) می\u200cباشد.\nزال در گذری از مرز کابل، به رودابه علاقه\u200cمند شد و او را به همسری برگزید.\nفرضیه: نام مادر رستم در شاهنامه، رودابه بود.\nجواب: <im_start>Assistant:آیا جمله دوم به جمله اول مرتبط است؟\nصحیح، غلط، شاید\nپیش فرض: رودابه نام دختر مهراب کابلی و سیندخت است او همسر زال و مادر رستم(شخصی

In [8]:
evall()

/tmp/ipykernel_2433421/3065205785.py:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric('rouge')
/home/hojjat/workstation/FarsInstruct/.venv/lib/python3.9/site-packages/datasets/load.py:855: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(
You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/hom

+---------------------------------------------------+
|                    Rouge Metric                   |
+-----------+------+-----------+--------+-----------+
|   Score   | Type | Precision | Recall | F-Measure |
+-----------+------+-----------+--------+-----------+
|   rouge1  | low  |   0.22    | 29.00  |    0.44   |
|   rouge1  | mid  |   0.26    | 32.50  |    0.52   |
|   rouge1  | high |   0.31    | 35.50  |    0.61   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rouge2  | low  |   0.00    |  0.00  |    0.00   |
|   rouge2  | mid  |   0.00    |  0.00  |    0.00   |
|   rouge2  | high |   0.00    |  0.00  |    0.00   |
|           |      |           |        |           |
|    ***    | ***  |    ***    |  ***   |    ***    |
|           |      |           |        |           |
|   rougeL  | low  |   0.23    | 29.24  |    0.45   |
|   rougeL  | mid  |   0.27 

(['سطح اطمینان خود را در شباهت عبارات ارائه شده بیان کنید\nعبارت اول: بُرهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nعبارت دوم: برهان فطرت نام دیگر برهان اتقان صنع است.\nسطح اطمینان: 0.8\nعبارت اول: برهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nعبارت دوم: برهان فطرت نام دیگر برهان اتقان صنع است.\nسطح اطمینان: 0.8\nعبارت اول: برهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nعبارت دوم: برهان فطرت نام دیگر برهان اتقان صنع است.\nسطح اطمینان: 0.8\nعبارت اول: برهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nعبارت دوم: برهان فطرت نام دیگر برهان اتقان صنع است.\nسطح اطمینان: 0.8\nعبارت اول: برهان نَظم یا برهان غایت\u200cشناختی یا اِتقان صُنع یا برهان الهیات طبیعی یکی از برهان\u200cهای اثبات وجود خدا است.\nعبارت دوم: برهان